In [1]:
import signalflow as sf
from signalflow.nn.validator import TemporalValidator
from pathlib import Path
from datetime import datetime
import polars as pl
import torch

spot_store = sf.data.raw_store.DuckDbSpotStore(db_path=Path("test.duckdb"))
raw_data = sf.data.RawDataFactory.from_duckdb_spot_store(
    spot_store_path=Path("test.duckdb"),
    pairs=["BTCUSDT", "ETHUSDT", "SOLUSDT"],
    start=datetime(2025, 10, 1),
    end=datetime(2025, 12, 31),
    data_types=["spot"],
)
raw_data_view = sf.core.RawDataView(raw_data)


feature_set = sf.feature.FeatureSet(extractors=[
    sf.feature.pandasta.PandasTaExtractor(kind="rsi", length=14),
    sf.feature.pandasta.PandasTaExtractor(kind="sma", length=20),
    sf.feature.pandasta.PandasTaExtractor(kind="atr", length=14),
])
features_df = feature_set.extract(raw_data_view)

features_df = features_df.with_columns([
    (pl.col(c) - pl.col(c).mean().over("pair")) / (pl.col(c).std().over("pair") + 1e-6)
    for c in features_df.columns if c not in ["pair", "timestamp"]
])

detector = sf.detector.MomentumDetector(threshold=0.01) 
signals = detector.run(raw_data_view)

from signalflow.target import FixedHorizonLabeler

labeler = FixedHorizonLabeler(price_col='close', horizon=12, threshold=0.015)
labeled_df = labeler.compute(raw_data_view.to_polars("spot"), signals)


def encode_labels(df: pl.DataFrame) -> pl.DataFrame:
    return df.with_columns(
        pl.when(pl.col("label") == 1).then(1)
          .when(pl.col("label") == -1).then(2)
          .otherwise(0)
          .cast(pl.Int64)
          .alias("label")
    )

train_signals_df = encode_labels(labeled_df).select(["pair", "timestamp", "label", "signal"])

train_signals_df = train_signals_df.filter(pl.col("label").is_not_null())



lstm_config = {
    "name": "encoder/lstm",
    "params": {
        "input_size": len(features_df.columns) - 2,
        "hidden_size": 64,
        "num_layers": 2,
        "dropout": 0.2,
        "bidirectional": False
    }
}

validator = TemporalValidator(
    encoder_config=lstm_config,
    window_size=30,           
    num_classes=3,          
    learning_rate=1e-3,
    max_epochs=10,            
    batch_size=64,
    train_val_test_split=(0.6, 0.2, 0.2)
)

print(f"Start Training Validator on {train_signals_df.height} signals...")


validator.fit(
    X_train=features_df, 
    y_train=train_signals_df,
    accelerator="auto" 
)

print("Training finished.")



validated_signals = validator.validate_signals(signals, features_df)


validated_signals_view = validated_signals.value.with_columns([
    pl.col("probability_class_1").alias("probability_rise"),
    pl.col("probability_class_2").alias("probability_fall"),
    pl.col("probability_class_0").alias("probability_neutral"),
])

print("\nTop Rise Signals:")
display(
    validated_signals_view
    .filter(pl.col("signal_type") == "rise")
    .sort("probability_rise", descending=True)
    .select(["timestamp", "pair", "probability_rise", "probability_fall"])
    .head()
)


/home/alastor/miniconda3/envs/sfnn/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'nn' is not defined